<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Data exploration and preprocessing
</div>

# 1. Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 2. Read data

In [2]:
raw_data = pd.read_csv('../data/data.csv')
raw_data

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,California,Female,Excellent,NaN,3.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.60,67.13,26.22,Yes,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
1,California,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,...,1.83,86.18,25.77,No,Yes,No,No,"Yes, received Tdap",No,No
2,California,Female,Very good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,No,8.0,1 to 5,No,...,1.63,55.79,21.11,No,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No
3,California,Female,Good,0.0,30.0,Within past year (anytime less than 12 months ...,No,7.0,None of them,No,...,1.75,72.57,23.63,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No
4,California,Male,Good,12.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,...,1.73,65.77,22.05,Yes,No,Yes,Yes,NaN,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10947,California,Male,Good,0.0,3.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.83,65.77,19.67,Yes,Yes,No,Yes,"Yes, received Tdap",No,No
10948,California,Male,Good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,8.0,None of them,No,...,1.78,90.72,28.70,Yes,NaN,Yes,Yes,NaN,No,Tested positive using home test without a heal...
10949,California,Female,Fair,5.0,5.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.57,68.04,27.44,No,No,Yes,NaN,"No, did not receive any tetanus shot in the pa...",No,Yes
10950,California,Female,Excellent,0.0,2.0,Within past 2 years (1 year but less than 2 ye...,Yes,10.0,None of them,No,...,1.75,58.97,19.20,No,Yes,No,NaN,"Yes, received Tdap",No,Yes


## Read data

In [ ]:
raw_data = pd.read_csv('../data/data.csv')
raw_data

## How many rows and how many columns does the raw data have?
- First, we should find out our dataset contains.

In [3]:
# Todo
raw_data.shape

(10952, 40)

We can see in this dataset, there are
- `10952 rows`
- `40 columns`

## What are the meanings of each row?

Each row in the dataset represents the **information of an individual** in the state of **California**, USA. The information includes the state, gender, general health status, number of days affected by physical and mental health, last health checkup time, physical activities, sleep hours, medical history, and other health-related factors.

## Does the raw data have duplicate rows? 

In [4]:
# Todo
def check_duplicates(data):
    duplicate_rows = data[data.duplicated()]
    if duplicate_rows.empty:
        print('No duplicate rows found')
    else:
        print('Duplicate rows found')
        print(duplicate_rows)
    return duplicate_rows
duplicates = check_duplicates(raw_data)

Duplicate rows found
           State     Sex GeneralHealth  PhysicalHealthDays  MentalHealthDays  \
2341  California    Male     Excellent                 0.0               0.0   
3474  California    Male     Excellent                 0.0               0.0   
6846  California  Female           NaN                 NaN               NaN   
9779  California    Male     Excellent                 0.0               0.0   

                                        LastCheckupTime PhysicalActivities  \
2341  Within past year (anytime less than 12 months ...                Yes   
3474  Within past year (anytime less than 12 months ...                Yes   
6846                                                NaN                NaN   
9779  Within past year (anytime less than 12 months ...                Yes   

      SleepHours  RemovedTeeth HadHeartAttack  ... HeightInMeters  \
2341         7.0  None of them             No  ...            NaN   
3474         7.0  None of them             No  ..

In this dataset, there are duplicated rows of data, so we will proceed to remove these rows.

In [5]:
# Remove duplicate rows
raw_data = raw_data.drop_duplicates()
# Check again
duplicate_rows =  check_duplicates(raw_data)

No duplicate rows found


The rows of data that have a significant number of missing attributes will be removed from the dataset.

In [6]:
threshold = 0.5 # Define the threshold percentage
# Calculate the percentage of missing values in each row
missing_percentage_row = raw_data.isnull().sum(axis = 1)/raw_data.shape[1]
# Check for rows with a significant percentage of missing data
rows_with_many_missing_values = raw_data[missing_percentage_row > threshold]
# Drop rows with a significant percentage of missing data
raw_data = raw_data.drop(rows_with_many_missing_values.index)
# Print a message after dropping the rows with a significant percentage of missing data
print(f"Dropped {len(rows_with_many_missing_values)} rows with a significant percentage of missing data.")

Dropped 76 rows with a significant percentage of missing data.


## What does each column mean?

In [7]:
raw_data.columns

Index(['State', 'Sex', 'GeneralHealth', 'PhysicalHealthDays',
       'MentalHealthDays', 'LastCheckupTime', 'PhysicalActivities',
       'SleepHours', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'AlcoholDrinkers',
       'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap',
       'HighRiskLastYear', 'CovidPos'],
      dtype='object')

Describe meaning of each column

In [ ]:
col_meaning_df = pd.read_csv('../data/schema.csv')
pd.set_option("display.max_colwidth", None)

col_meaning_df

## What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?

In [8]:
raw_data.dtypes

State                         object
Sex                           object
GeneralHealth                 object
PhysicalHealthDays           float64
MentalHealthDays             float64
LastCheckupTime               object
PhysicalActivities            object
SleepHours                   float64
RemovedTeeth                  object
HadHeartAttack                object
HadAngina                     object
HadStroke                     object
HadAsthma                     object
HadSkinCancer                 object
HadCOPD                       object
HadDepressiveDisorder         object
HadKidneyDisease              object
HadArthritis                  object
HadDiabetes                   object
DeafOrHardOfHearing           object
BlindOrVisionDifficulty       object
DifficultyConcentrating       object
DifficultyWalking             object
DifficultyDressingBathing     object
DifficultyErrands             object
SmokerStatus                  object
ECigaretteUsage               object
C

- In general, the data types of the columns are not unusual, but for convenience in the working process, we will convert the data types of some columns and drop unnecessary columns.

With each numerical column, how are values distributed?
- What is the percentage of missing values?
- If there are missing values, handle them.
- Min? max? Are they abnormal?
- Missing value treatment.

### Select numeric columns

In [ ]:
num_col_df = raw_data.select_dtypes(include='float64')
num_col_df

### Explore the distribution using descriptive statisticsWith each numerical column, how are values distributed?
- What is the percentage of missing values?
- If there are missing values, handle them.
- Min? max? Are they abnormal?
- Missing value treatment

In [ ]:
def missing_ratio(col):
    return (col.isna().sum() * 100 / len(col)).round(1)

def lower_quartile(col):
    return col.quantile(0.25).round(1)

def upper_quartile(col):
    return col.quantile(0.75).round(1)

num_col_info_df = num_col_df.agg([missing_ratio, 'min', lower_quartile, 'median', upper_quartile, 'max'])
num_col_info_df

**Observation:**
- The percentage of missing values of each numeric column is low so we won't drop any of these columns. Instead, we try to handle these missing values.
- The minimum and maximum values of each numeric column are within normal ranges:
    - There are no negative numbers.
    - PhysicalHealthDays and MentalHealthDays both have values equal or less than 30.
    - SleepHours have values equal or less than 24.
    - Three remaining columns also have reasonable values.
- Based on upper-quartile values and max values, we can see PhysicalHealthDays, MentalHealthDays, SleepHours, WeightInKilograms and BMI have right-skewed distributions as upper-quartile values are far from max values.
- Because of that, we will fill missing values in these columns with the median (an indicator that is insensitive with outliers).

### Visualize missing ratio

In [ ]:
data = num_col_info_df.loc['missing_ratio']
fig = px.bar(x=data.index, y=data.values, width=1000, height=500, labels={'x': 'Numeric column', 'y': 'Percentage(%)'}, 
             title='Missing ratio of numeric columns')
fig.show()

### Handle missing values

In [ ]:
raw_data[num_col_df.columns] = raw_data[num_col_df.columns].fillna(num_col_df.median())

After handling missing values, we will check missing-ratio again to ensure that we have handle missing values successfully.

In [ ]:
non_nan_num_cols = raw_data[num_col_df.columns] 
non_nan_num_col_info_df = non_nan_num_cols.agg([missing_ratio, 'min', lower_quartile, 'median', upper_quartile, 'max'])
non_nan_num_col_info_df

Now there are no missing values. 

### Visualize the distribution 

We will use **histogram** to visualize the distributions of numeric columns and provide insights we can gain from them.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 6))
axes = axes.flatten()
plt.subplots_adjust(hspace=0.4)

bin_nums = [10, 10, 23, 20, 20, 20]
for i in range(len(axes)):
    axes[i].hist(raw_data[non_nan_num_cols.columns[i]], bins=bin_nums[i])
    axes[i].set_title(non_nan_num_cols.columns[i]);

**Observation**:
- The physical health of people in California is generally good, as most of them experienced physical health problems for less than 6 days.
- The number of people experiencing mental health problems for more than 6 days is quite higher than that of physical health problems. It can be observed that mental health problems often persist for a longer duration compared to physical health problems.
- The distribution of the SleepHours column is narrow, indicating that most people in California have average sleep hours around 6 to 9 hours per day, which is good for health.
- The height of people in California is various, but focused mainly in the range of 1.5 to 1.85 meters.
- Similarly, the weight and BMI are primarily centered in the range of 50 to 112 kilograms and 20 to 35.

### Handle outliers

First, we will see if there are any outliers in numerical columns.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 6))
axes = axes.flatten()
plt.subplots_adjust(hspace=0.3)

non_nan_num_cols = raw_data[num_col_df.columns]
for i in range(len(axes)):
    axes[i].boxplot(non_nan_num_cols.iloc[:, i]);
    axes[i].set_title(non_nan_num_cols.columns[i])

**With each catagorical column, how are values distributed?**

- What is the percentage of missing values?

*Select catagorical columns*

In [ ]:
# Select catagorical columns in data
cat_col_df = raw_data.select_dtypes(include = ['object'])
cat_col_df.head(5)

In [ ]:
# Function 
def missing_ratio(col):
    return (col.isnull().mean()*100).round(2)

# Calculate missing ratio for catagorical columns
cat_missing_ratio = cat_col_df.agg(missing_ratio)
# Print the result
print("The percentage of missing values:")
for col, ratio in cat_missing_ratio.items():
    print(f"- {col}: {ratio}%")

*Visualize missing ratio of catagorical columns*

In [ ]:
cat_missing_ratio_df = pd.DataFrame({'Column': cat_missing_ratio.index, 'Missing Ratio (%)': cat_missing_ratio.values})
fig = px.bar(cat_missing_ratio_df, x = cat_missing_ratio_df['Missing Ratio (%)'], y = 'Column', 
             title = 'Missing Ratio of Catagorical Columns', range_x= (0, 100),
             text = 'Missing Ratio (%)')
fig.update_layout(height=800, width=800)
fig.show()

**Observation:**
- The missing value ratio of each catagorical column is quite low, so we will keep the columns and handle the missing values.

- How many different values? Are they abnormal?

In [ ]:
cat_col_info_df = cat_col_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

## Outliers detection